In [1]:
first_name= "Pouria"
last_name= "Dadkhah"
student_number= "401201381"

In [2]:
from google.colab import drive
drive.mount('/content/drive')
ferdousi_path = '/content/drive/MyDrive/Deep Learning/DL_HW4/Q3 Materials/ferdousi.txt'

Mounted at /content/drive


In [3]:
import torch
import torchtext
from torchtext.data import get_tokenizer
from sklearn.model_selection import train_test_split
import pandas as pd

# Set the seed for reproducibility
SEED = 80
torch.manual_seed(SEED)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')

# Tokenizer
tokenizer = get_tokenizer("basic_english")

# Data Preprocessing
def text_to_data(path):
    lines = []
    with open(path, 'r', encoding='utf-8') as file:
        for row in file:
            lines.append(row.strip())

    poem_dict = {'source': [], 'target': []}

    # Ensure the length is a multiple of 4
    len_lines = len(lines) - len(lines) % 4

    for i in range(4, len_lines, 4):
        source_line = lines[i-2].strip() + ' / ' + lines[i-1].strip()
        target_line = lines[i].strip() + ' / ' + lines[i+1].strip()

        poem_dict['source'].append(source_line)
        poem_dict['target'].append(target_line)

    return pd.DataFrame(poem_dict)

ferdousi_path = '/content/drive/MyDrive/Deep Learning/DL_HW4/Q3 Materials/ferdousi.txt'
df = text_to_data(ferdousi_path)

# Train-Test Split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=SEED)
train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

# Display some samples
print("Training Data Sample:")
print(train_df.head())

print("\nTesting Data Sample:")
print(test_df.head())
df.head()

Device: cpu
Training Data Sample:
                                              source  \
0  همان کوه کو کرده دارد حصار / باسیان جنگی ز پا ...   
1  تو خودکامه را گر ندانی شمار / بروچارصد بار بشم...   
2  بپیچید و بر زد یکی سرد باد / به زاری بران جامه...   
3  ورا مژده دادی به فرزند او / بران برز شاخ برومن...   
4  همه جنگ را تیغها برکشید / وزین دشت هامون سر ان...   

                                              target  
0   مکش دست ازیشان بخون ریختن / تو پیروز باشی بویختن  
1  زپیلان جنگی هزار و دویست / که گفتی که بر راه ب...  
2  برین گونه گردد جهان جهان / همی راز خویش از تو ...  
3  چو بیدار شد موبدان را بخواند / ازین در سخن چند...  
4  چو در جنگ تن را به رنج آورید / ازان رنج شاهی و...  

Testing Data Sample:
                                              source  \
0  خردنیست با گرد گردان سپهر / نه پیدابود رنج و خ...   
1  ز ایران بیامد دمادم سپاه / ز راه بیابان سوی رز...   
2  تو از من به سال اندکی برتری / تو باید که چون م...   
3  چو اسفندیار آن گو تهمتن / خداوند اورنگ با سهم ... 

,source,target
0,به نام خداوند جان و خرد / کزین برتر اندیشه برن...,خداوند نام و خداوند جای / خداوند روزی ده رهنمای
1,خداوند کیوان و گردان سپهر / فروزنده ماه و ناهی...,ز نام و نشان و گمان برترست / نگارندهٔ بر شده پ...
2,به بینندگان آفریننده را / نبینی مرنجان دو بینن...,نیابد بدو نیز اندیشه راه / که او برتر از نام و...
3,سخن هر چه زین گوهران بگذرد / نیابد بدو راه جان...,خرد گر سخن برگزیند همی / همان را گزیند که بیند...
4,ستودن نداند کس او را چو هست / میان بندگی را بب...,خرد را و جان را همی سنجد اوی / در اندیشهٔ سخته...


In [4]:
!pip install transformers

In [5]:
import torch
from transformers import GPT2Tokenizer
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from sklearn.model_selection import train_test_split
import pandas as pd

# Set the seed for reproducibility
SEED = 80
torch.manual_seed(SEED)

# Tokenizer for Persian language from Hugging Face Transformers
tokenizer_fa = GPT2Tokenizer.from_pretrained("HooshvareLab/gpt2-fa")

# Vocabulary class
class Vocabulary():
    def __init__(self, freq_threshold):
        self.itos = {0: "", 1: "", 2: "", 3: ""}
        self.stoi = {"": 0, "": 1, "": 2, "": 3}
        self.freq_threshold = freq_threshold

    def __len__(self):
        return len(self.itos)

    def tokenizer_fa(self, text):
        return tokenizer_fa.tokenize(text)

    def build_vocabulary(self, sentence_list):
        frequencies = {}
        idx = 4
        for sentence in sentence_list:
            for word in self.tokenizer_fa(sentence):
                if word not in frequencies:
                    frequencies[word] = 1
                else:
                    frequencies[word] += 1

                if frequencies[word] == self.freq_threshold:
                    self.stoi[word] = idx
                    self.itos[idx] = word
                    idx += 1

    def numericalize(self, text):
        tokenized_text = self.tokenizer_fa(text)

        return [
            self.stoi[token] if token in self.stoi else self.stoi[""]
            for token in tokenized_text
        ]

# Ferdousi Dataset class
class FerdousiDataset(Dataset):
    def __init__(self, threshold=2, train_df=None, test_df=None, vocabulary=None):

        self.train_df = train_df
        self.test_df = test_df

        if self.train_df is not None:
            self.data = self.train_df
        else:
            self.data = self.test_df

        self.data.rename(columns={'source': 'Input', 'target': 'Output'}, inplace=True)

        # Initialize vocabulary and build vocab
        self.vocab = vocabulary
        input_list = self.data['Input'].to_list()
        output_list = self.data['Output'].to_list()
        input_list.extend(output_list)

        if self.train_df is not None:
            self.vocab.build_vocabulary(input_list)

        self.dictionary_size = len(self.vocab.itos)
        self.PAD_IDX = self.vocab.stoi['']

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        inp = self.data.loc[index, 'Input']
        out = self.data.loc[index, 'Output']

        numericalized_input = [self.vocab.stoi[""]]
        numericalized_input += self.vocab.numericalize(inp)
        numericalized_input.append(self.vocab.stoi[""])

        numericalized_output = [self.vocab.stoi[""]]
        numericalized_output += self.vocab.numericalize(out)
        numericalized_output.append(self.vocab.stoi[""])

        sample = {
            'Input': torch.tensor(numericalized_input),
            'Output': torch.tensor(numericalized_output)
        }

        return sample

# Create vocabulary and datasets
threshold = 2
Ferdousi_Vocab = Vocabulary(threshold)

# Train-Test Split
train_dataset = FerdousiDataset(train_df=train_df, vocabulary=Ferdousi_Vocab)
test_dataset = FerdousiDataset(test_df=test_df, vocabulary=Ferdousi_Vocab)

dictionary_size = train_dataset.dictionary_size
print(f'Number of words in Ferdousi dictionary: {dictionary_size:,}')

print(f"Number of training examples: {len(train_dataset):,}")
print(f"Number of testing examples: {len(test_dataset):,}")

# # Dataloader
# class MyCollate():
#     def __init__(self, pad_idx):
#         self.pad_idx = pad_idx

#     def __call__(self, batch):
#         inp = [item['Input'] for item in batch]
#         out = [item['Output'] for item in batch]
#         Input = pad_sequence(inp, batch_first=False, padding_value=self.pad_idx)
#         Output = pad_sequence(out, batch_first=False, padding_value=self.pad_idx)

#         return Input, Output

# Modify your DataLoader to return input and target batches separately
class MyCollate():
    def __init__(self, pad_idx):
        self.pad_idx = pad_idx

    def __call__(self, batch):
        inp = [item['Input'] for item in batch]
        out = [item['Output'] for item in batch]

        # Pad sequences to the maximum length in the batch
        max_len_inp = max(len(seq) for seq in inp)
        max_len_out = max(len(seq) for seq in out)

        inp = pad_sequence(inp, batch_first=False, padding_value=self.pad_idx)
        out = pad_sequence(out, batch_first=False, padding_value=self.pad_idx)

        inp = inp[:, :max_len_inp]  # Truncate to the maximum length
        out = out[:, :max_len_out]

        return inp, out


BATCH_SIZE = 128
pad_idx = train_dataset.vocab.stoi[""]


# Update your DataLoader instantiation
# train_dataloader = DataLoader(dataset=train_dataset,
#                               batch_size=BATCH_SIZE,
#                               shuffle=True,
#                               collate_fn=MyCollate(pad_idx=pad_idx))

train_dataloader = DataLoader(dataset=train_dataset,
                              batch_size=BATCH_SIZE,
                              shuffle=True,
                              collate_fn=MyCollate(pad_idx=pad_idx))
test_dataloader = DataLoader(test_dataset,
                             batch_size=BATCH_SIZE,
                             shuffle=False,
                             collate_fn=MyCollate(pad_idx=pad_idx))


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/728 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/875k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/14.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/104 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.75M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/808 [00:00<?, ?B/s]

Number of words in Ferdousi dictionary: 5,905
Number of training examples: 19,842
Number of testing examples: 4,961


In [6]:
import torch
import torch.nn as nn
from transformers import GPT2LMHeadModel, GPT2Config, GPT2Tokenizer
from torch.optim import Adam

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load pre-trained GPT-2 model and tokenizer
model_name = "HooshvareLab/gpt2-fa"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name).to(device)

# Define the loss function
loss_function = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)

# Optimizer
optimizer = Adam(model.parameters(), lr=1e-4)

# Training loop
def train(model, train_dataloader, loss_function, optimizer, num_epochs=5):
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0.0
        for input_batch, target_batch in train_dataloader:
            input_batch, target_batch = input_batch.to(device), target_batch.to(device)

            # Ensure the target_batch does not include padding
            target_batch = target_batch[:, 1:].contiguous().view(-1)

            optimizer.zero_grad()
            outputs = model(input_ids=input_batch, labels=target_batch)
            loss = outputs.loss
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        average_loss = total_loss / len(train_dataloader)
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {average_loss:.4f}")

# Training the model
num_epochs = 5  # You can adjust this number based on your needs
train(model, train_dataloader, loss_function, optimizer, num_epochs)


pytorch_model.bin:   0%|          | 0.00/485M [00:00<?, ?B/s]

ValueError: Expected input batch_size (462) to match target batch_size (419).

we have struggled through working with data loader size matching but didn't make it through the pre-trained model unfortunatly but the rest of the way is as implemented through the notebook